# Measurement Setup

In [ ]:
#imports
import matplotlib.pyplot as plt
import numpy as np
import os
import random

In [ ]:
#XMEGA
PLATFORM = 'CW303'

In [ ]:
#STM 32
PLATFORM = 'CW308_STM32F3'

In [ ]:
SCOPETYPE = 'OPENADC'
CRYPTO_TARGET = 'CHACHA'
CHUNK_SIZE = 10_000

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/simpleserial-chacha
make PLATFORM=$1 CRYPTO_TARGET=$2
# make chacha.s PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../hardware/victims/firmware/simpleserial-chacha/simpleserial-chacha-{}.hex'.format(PLATFORM)
cw.program_target(scope, prog, fw_path)

In [ ]:
scope.clock

In [ ]:
scope.adc

In [ ]:
target.baud = 230400

In [ ]:
target.simpleserial_write('x', [])

In [ ]:
print(target.simpleserial_wait_ack()) #should return 0

# Experiment - duration of 10, 100, 1000,... encryptions

In [ ]:
import os

# helper functions
# - to create and send key
def setup_key():
    # 256bit-long key => 32 bytes
    k = os.urandom(32)
    key_cmd = "k" + k.hex() + "\n"
    target.write(key_cmd)
    target.simpleserial_wait_ack()
    return k
# - to create and send Nonce
def setup_nonce():
    # 96bit-long key => 12 bytes
    n = os.urandom(12)
    nonce_cmd = "n" + n.hex() + "\n"
    target.write(nonce_cmd)
    target.simpleserial_wait_ack()
    return n
# - to create and send PT
def encrypt_random_PT():
    # 612bit-long PT => 64 bytes
    pt = os.urandom(64) 
    encrypt_cmd = "p" + pt.hex() + "\n"
    target.write(encrypt_cmd)
    target.read()
    return pt
    
class Context:
    key        = None
    nonce      = None
    plaintext  = None
    ciphertext = None
    
    def __init__(self, key=None, nonce=None, plaintext=None, ciphertext=None):
        self.key = key
        self.nonce = nonce
        self.plaintext = plaintext
        self.ciphertext = ciphertext

    def __str__(self):
        return (f"Context:\n"
                f"  Key:        {self.key.hex()}\n"
                f"  Nonce:      {self.nonce.hex()}\n"
                f"  Plaintext:  {self.plaintext.hex()}\n"
                f"  Ciphertext: {self.ciphertext.hex()}")
    
def save_context_to_file(context, folder, file_suffix = ""):
    with open(f'{folder}/keys_{file_suffix}.bin', 'wb') as f:
        for item in context:
                f.write(item.key)
    
    with open(f'{folder}/nonces_{file_suffix}.bin', 'wb') as f:
        for item in context:
                f.write(item.nonce)
    
    with open(f'{folder}/plaintexts_{file_suffix}.bin', 'wb') as f:
        for item in context:
                f.write(item.plaintext)
    
    with open(f'{folder}/ciphertexts_{file_suffix}.bin', 'wb') as f:
        for item in context:
                f.write(item.ciphertext)
            
def save_info_to_file(num, trace_len, duration, file_name):
    with open(file_name, 'w') as f:
        f.write(f"{num}\n{trace_len}\n{duration}\n")
            
# function to perform encryption and measure duration
def capture_traces(num, trace_len = 10_000, file_name_suffix = ""):
    traces = []
    context_array = []
    scope.adc.samples = trace_len
    start_time = time.time()
    for _ in range(num):
        context = Context()
        
        context.nonce = setup_nonce()
        context.key   = setup_key()

        scope.arm()

        context.plaintext = encrypt_random_PT()    
        ret = scope.capture()
        if ret:
            print("Target timed out!")
            continue

        context.ciphertext = target.simpleserial_read('r', 64)
        adc_array_uint16 = np.array(scope.get_last_trace(True), dtype=np.uint16)
        traces.append(adc_array_uint16)
        context_array.append(context)
    
    if not os.path.exists("ChaCha20"):
        os.makedirs("ChaCha20")
        print(f"DIR ChaCha20 created")
        
    save_context_to_file(context_array, "ChaCha20", f"{num}_{file_name_suffix}")
    traces = np.array(traces)
    traces.tofile(f"ChaCha20/traces_{num}_{file_name_suffix}.bin")
    
    end_time = time.time()
    duration = end_time - start_time
    
    save_info_to_file(num, trace_len, duration, f"ChaCha20/info_{num}_{file_name_suffix}.txt")
    
    return duration, traces

In [ ]:
duration, traces = capture_traces(2, 21304, "test_111")
# Plot the traces
plt.figure(figsize=(10, 5))
for trace in traces:
    plt.plot(trace, 'k.', markersize=1)  # Black dots for each trace

# Add labels and title
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.title(f'Traces (Length: {traces.shape[1]})')

# Show the plot
plt.show()

In [ ]:
#necessary imports
import time
import os

# target set-up 
# - baud-rate to 230400
# target.baud = 230400
time.sleep(1)
print(f"Baud-rate = {target.baud}")


# scope.adc.trig_count
counts = [10, 100, 1000, 10_000]
results = {}

# main loop to process all enryption counts
for count in counts:
    duration, _ = capture_traces(count, trace_len = 4056, file_name_suffix = target.baud)
    results[count] = duration
    print(f"Encryption count: {count}, Total duration: {duration}, Average duration: {duration / count} seconds")
    


# Helper functions

In [ ]:
PT_GLOBAL = None
def setup_fixed_pt():
    global PT_GLOBAL
    pt = os.urandom(64) 
    PT_GLOBAL = "p" + pt.hex() + "\n"
    return pt
    
def encrypt_fixed_PT():
    global PT_GLOBAL
    target.write(PT_GLOBAL)
    target.read()
    
def save_array_to_bin(array, file_name):
    with open(file_name, 'wb') as f:
        for item in array:
            f.write(item)
            
def save_fixed_parameters(key, nonce, plaintext_fixed, ciphertext_fixed, folder):
    save_array_to_bin([key], f"{folder}/key.bin")
    save_array_to_bin([nonce], f"{folder}/nonce.bin")
    save_array_to_bin([plaintext_fixed], f"{folder}/plaintext_fixed.bin")
    save_array_to_bin([ciphertext_fixed], f"{folder}/ciphertext_fixed.bin")

def save_info_to_file_TTest(num, chunk_size, trace_len, random_cnt, fixed_cnt, duration, file_name):
    with open(file_name, 'w') as f:
        f.write(f"{num}\n{chunk_size}\n{trace_len}\n{random_cnt}\n{fixed_cnt}\n{duration}\n")
        
def load_nonce_from_file(file_path):
    with open(file_path, 'rb') as f:
        nonce = f.read(12)
        if len(nonce) != 12:
            raise ValueError("None must be 12 bytes")
    return nonce

def load_key_from_file(file_path):
    with open(file_path, 'rb') as f:
        key = f.read(32)
        if len(key) != 32:
            raise ValueError("Key must be 32 bytes")
    return key

def save_array_to_bin(array, file_name):
    with open(file_name, 'wb') as f:
        for item in array:
            f.write(item)
            
def save_fixed_parameters_CPA(folder, key, nonce = []):
    save_array_to_bin([key], f"{folder}/key.bin")
    if nonce:
        save_array_to_bin([nonce], f"{folder}/nonce.bin")

def save_info_to_file_CPA(num, chunk_size, trace_len, duration, file_name):
    with open(file_name, 'w') as f:
        f.write(f"{num}\n{chunk_size}\n{trace_len}\n{duration}\n")

# Capture traces for t-test

In [ ]:
def collect_data_for_t_test(count, folder, trace_len =10000, fixed_nonce = True, chunk_index = 0, load_from_file = False):
    global CHUNK_SIZE
    chunk_random_index = chunk_index
    chunk_fixed_index  = chunk_index
    print(f"Baud-rate = {target.baud}")
    
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"DIR {folder} created")
            
    if load_from_file:
        key = load_key_from_file(folder + "/key.bin")
    else:
        key = setup_key()
    
    plaintext_fixed  = setup_fixed_pt()
    ciphertext_fixed = None
    ciphertext_fixed = setup_fixed_pt()
    traces_fixed     = []
    
    nonce = None
    
    #save the key and nonce as well for possible CPA analysis    
    if fixed_nonce:
        nonce = setup_nonce()
        save_fixed_parameters_CPA(folder, key, nonce)
    else:
        nonces_random = []
        nonces_fixed  = []
        save_fixed_parameters_CPA(folder, key)

    traces_random      = []
    ciphertexts_random = []
    plaintexts_random  = []
    
    fixed_cnt  = chunk_index * CHUNK_SIZE
    random_cnt = chunk_index * CHUNK_SIZE
    
    scope.adc.samples = trace_len
    start_time = time.time()
    while fixed_cnt < count or random_cnt < count:
        probabily = random.random()
        encrypt_fixed_pt = probabily < 0.5
        
        if not fixed_nonce:
            nonce = setup_nonce()
        
        scope.arm()
        # Generate a random number and choose an action based on its value
        if encrypt_fixed_pt:
            encrypt_fixed_PT() 
        else:
            plaintexts_random.append(encrypt_random_PT())
            
        ret = scope.capture()
        if ret:
            print("Target timed out!")
            continue
        
        try:
            ciphertext = target.simpleserial_read('r', 64)
            if ciphertext is None:
                raise ValueError("Received None as ciphertext")
        except Exception as e:
            print(f"Error reading ciphertext ~{fixed_cnt}: {e}")
            if not encrypt_fixed_pt:
                plaintexts_random.pop()  # Remove the last plaintext since capture failed
            continue
        
        adc_array_uint16 = np.array(scope.get_last_trace(True), dtype=np.uint16)
        
        if encrypt_fixed_pt:
            traces_fixed.append(adc_array_uint16)
            if ciphertext_fixed is None:
                ciphertext_fixed = ciphertext
            if not fixed_nonce:
                nonces_fixed.append(nonce)
            fixed_cnt += 1
        else:
            traces_random.append(adc_array_uint16)
            ciphertexts_random.append(ciphertext)
            if not fixed_nonce:
                nonces_random.append(nonce)
            random_cnt += 1
#------------
            
        # Save every CHUNK_SIZE traces
        if random_cnt % CHUNK_SIZE == 0 and traces_random:
            print(f"Creating Random Data Chunk {chunk_random_index}...")
            chunk_folder = os.path.join(folder, f"chunk_{chunk_random_index}")
            
            if not os.path.exists(chunk_folder):
                os.makedirs(chunk_folder)

            traces_random_np = np.array(traces_random)
            traces_random_np.tofile(f"{chunk_folder}/traces_random.bin")

            save_array_to_bin(plaintexts_random, f"{chunk_folder}/plaintexts_random.bin")
            save_array_to_bin(ciphertexts_random, f"{chunk_folder}/ciphertexts_random.bin")
            
            if not fixed_nonce:
                save_array_to_bin(nonces_random, f"{chunk_folder}/nonces_random.bin")
                nonces_random.clear()

            # Clear the lists for the next chunk
            traces_random.clear()
            ciphertexts_random.clear()
            plaintexts_random.clear()

            chunk_random_index += 1
        
        # Save every CHUNK_SIZE traces
        if fixed_cnt % CHUNK_SIZE == 0 and traces_fixed:
            print(f"Creating Fixed Data Chunk {chunk_fixed_index}...")
            chunk_folder = os.path.join(folder, f"chunk_{chunk_fixed_index}")
            if not os.path.exists(chunk_folder):
                os.makedirs(chunk_folder)

            traces_fixed_np = np.array(traces_fixed)
            traces_fixed_np.tofile(f"{chunk_folder}/traces_fixed.bin")
            
            if not fixed_nonce:
                save_array_to_bin(nonces_fixed, f"{chunk_folder}/nonces_fixed.bin")
                nonces_fixed.clear()

            # Clear the lists for the next chunk
            traces_fixed.clear()
            
            chunk_fixed_index += 1
    
    # END OF: while fixed_cnt < count or random_cnt < count
    
    # Save any remaining traces if count is not a multiple of chunk_size
    if traces_random:
        print(f"Creating Random Data Chunk {chunk_random_index}...")
        chunk_folder = os.path.join(folder, f"chunk_{chunk_random_index}")
        if not os.path.exists(chunk_folder):
            os.makedirs(chunk_folder)
        
        traces_random_np = np.array(traces_random)
        traces_random_np.tofile(f"{chunk_folder}/traces_random.bin")
        
        save_array_to_bin(plaintexts_random, f"{chunk_folder}/plaintexts_random.bin")
        save_array_to_bin(ciphertexts_random, f"{chunk_folder}/ciphertexts_random.bin")
        
        if not fixed_nonce:
                save_array_to_bin(nonces_random, f"{chunk_folder}/nonces_random.bin")
        
    if traces_fixed:
        print(f"Creating Fixed Data Chunk {chunk_fixed_index}...")
        chunk_folder = os.path.join(folder, f"chunk_{chunk_fixed_index}")
        if not os.path.exists(chunk_folder):
            os.makedirs(chunk_folder)
        
        traces_fixed_np = np.array(traces_fixed)
        traces_fixed_np.tofile(f"{chunk_folder}/traces_fixed.bin")
        
        if not fixed_nonce:
            save_array_to_bin(nonces_fixed, f"{chunk_folder}/nonces_fixed.bin")

    end_time = time.time()
    duration = end_time - start_time
    print (f"duration: {duration}, random_cnt: {random_cnt}, fixed_cnt: {fixed_cnt}")
    save_info_to_file_TTest(count, CHUNK_SIZE, trace_len, random_cnt, fixed_cnt, duration, f"{folder}/info.txt")

In [ ]:
scope.adc.trig_count

In [ ]:
CHUNK_SIZE = 10_000
# def collect_data_for_t_test(count, folder, trace_len =10000, fixed_nonce = True, chunk_index = 0, load_from_file = False):
# collect_data_for_t_test(1_000_000, "ChaCha-1000-000-Random-Nonce-XMEGA", 21304, False)
# collect_data_for_t_test(1_000_000, "ChaCha-1000-000-Random-Nonce-STM",    9168, False)
# collect_data_for_t_test(1_000_000, "ChaCha-1000-000-Random-Nonce-XMEGA-2", 21304, False)
# collect_data_for_t_test(1_000_000, "ChaCha-1000-000-Random-Nonce-STM-2",    9168, False)


# Capture traces for CPA

In [ ]:
import random
def collect_data_for_CPA(count, folder, trace_len = 10000, fixed_nonce = True, chunk_index = 0, load_from_file = False):
    global CHUNK_SIZE
    print(f"Baud-rate = {target.baud}")
    
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"DIR {folder} created")
        
    if not fixed_nonce:
        raise "Not implemented yet..."
    elif load_from_file:
        nonce = load_nonce_from_file(folder + "/nonce.bin")
    else:
        nonce = setup_nonce()
    
    if load_from_file:
        key = load_key_from_file(folder + "/key.bin")
    else:
        key = setup_key()
    
    traces_random      = []
    ciphertexts_random = []
    plaintexts_random  = []
    
    random_cnt = 0
    
    scope.adc.samples = trace_len
    start_time = time.time()
    
    save_fixed_parameters_CPA(folder, key, nonce)
    
    while random_cnt < count:
        scope.arm()
        plaintexts_random.append(encrypt_random_PT())
        ret = scope.capture()
        if ret:
            print("Target timed out!")
            continue
        
        try:
            ciphertext = target.simpleserial_read('r', 64)
            if ciphertext is None:
                raise ValueError("Received None as ciphertext")
        except Exception as e:
            print(f"Error reading ciphertext: {e}")
            plaintexts_random.pop()  # Remove the last plaintext since capture failed
            continue
            
        adc_array_uint16 = np.array(scope.get_last_trace(True), dtype=np.uint16)
        traces_random.append(adc_array_uint16)
        ciphertexts_random.append(ciphertext)
        random_cnt += 1
    
        # Save every chunk_size traces
        if random_cnt % CHUNK_SIZE == 0:
            print(f"Creating Chunk {chunk_index}...")
            chunk_folder = os.path.join(folder, f"chunk_{chunk_index}")
            if not os.path.exists(chunk_folder):
                os.makedirs(chunk_folder)

            traces_random_np = np.array(traces_random)
            traces_random_np.tofile(f"{chunk_folder}/traces_random.bin")

            save_array_to_bin(plaintexts_random, f"{chunk_folder}/plaintexts_random.bin")
            save_array_to_bin(ciphertexts_random, f"{chunk_folder}/ciphertexts_random.bin")

            # Clear the lists for the next chunk
            traces_random.clear()
            ciphertexts_random.clear()
            plaintexts_random.clear()

            chunk_index += 1
            
            end_time = time.time()
            duration = end_time - start_time
            print (f"Duration: {duration}")
    

     # Save any remaining traces if count is not a multiple of chunk_size
    if traces_random:
        print(f"Creating Last Chunk {chunk_index}...")
        chunk_folder = os.path.join(folder, f"chunk_{chunk_index}")
        if not os.path.exists(chunk_folder):
            os.makedirs(chunk_folder)
        
        traces_random_np = np.array(traces_random)
        traces_random_np.tofile(f"{chunk_folder}/traces_random.bin")
        
        save_array_to_bin(plaintexts_random, f"{chunk_folder}/plaintexts_random.bin")
        save_array_to_bin(ciphertexts_random, f"{chunk_folder}/ciphertexts_random.bin")
    
    end_time = time.time()
    duration = end_time - start_time
    print (f"Duration: {duration}")
    save_info_to_file_CPA(count, CHUNK_SIZE, trace_len, duration, f"{folder}/info.txt")